In [1]:
import pandas as pd
import requests

def dow_jones_symbols():
    """
    Returns a list of the 30 stocks in the Dow Jones.

    Returns
    -------
    list
        a list of strings that are stock symbols which make up the Dow Jones Industrial Average
    """

    headers = {
        'authority': 'quote.cnbc.com',
        'accept': '*/*',
        'accept-language': 'en-US,en;q=0.9',
        'origin': 'https://www.cnbc.com',
        'referer': 'https://www.cnbc.com/',
        'sec-ch-ua': '"Not?A_Brand";v="8", "Chromium";v="108", "Google Chrome";v="108"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-site',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    }

    params = {
        'symbols': 'AXP|AMGN|AAPL|BA|CAT|CSCO|CVX|GS|HD|HON|IBM|INTC|JNJ|KO|JPM|MCD|MMM|MRK|MSFT|NKE|PG|TRV|UNH|CRM|VZ|V|WBA|WMT|DIS|DOW',
        'requestMethod': 'itv',
        'noform': '1',
        'partnerId': '2',
        'fund': '1',
        'exthrs': '1',
        'output': 'json',
        'events': '1',
    }
    
    response = requests.get(
        'https://quote.cnbc.com/quote-html-webservice/restQuote/symbolType/symbol',
        params=params,
        headers=headers,
    )
    data = response.json()['FormattedQuoteResult']['FormattedQuote']
    
    symbols = []
    for i in data:
        symbol = i['symbol']
        symbols.append(symbol)

    dj = pd.DataFrame(symbols, columns = ['Symbol'])
    dj = dj.sort_values(by=['Symbol'])
    dow_jones_list = []
    for i in dj['Symbol']:
        dow_jones_list.append(i)

    return dow_jones_list

In [2]:
dow_jones_symbols()

['AAPL',
 'AMGN',
 'AXP',
 'BA',
 'CAT',
 'CRM',
 'CSCO',
 'CVX',
 'DIS',
 'DOW',
 'GS',
 'HD',
 'HON',
 'IBM',
 'INTC',
 'JNJ',
 'JPM',
 'KO',
 'MCD',
 'MMM',
 'MRK',
 'MSFT',
 'NKE',
 'PG',
 'TRV',
 'UNH',
 'V',
 'VZ',
 'WBA',
 'WMT']

In [3]:
def debt_to_equity(df, symbol=""):
    for j in df:
        if j == 'ticker' or j == 'items':
            continue
        df[j] = df[j].str.replace(",", "", regex=True).str.replace("(", "-", regex=True).str.replace(")", "", regex=True).str.replace("--", "", regex=True)
        df[j] = pd.to_numeric(df[j])\

    df_columns = []

    for col in df.columns:
        df_columns.append(col)

    df_columns.insert(1, df_columns[-1])
    del df_columns[-1]

    first_row = df.loc[(df['ticker'] == symbol) & (df['items'] == 'Total Liabilities'), df_columns[2:]]
    second_row = df.loc[(df['ticker'] == symbol) & (df['items'] == 'Total Equity'), df_columns[2:]]
#     print(first_row)
#     print(second_row)
    third_row = first_row / second_row.values
#     print(third_row)
    third_row['ticker'] = symbol
    third_row['items'] = "Debt-to-Equity Ratio"
    third_row = third_row[df_columns]
    return third_row

In [4]:
import os
from os import path

dow_jones_list = dow_jones_symbols()

for i in dow_jones_list:
    df = pd.DataFrame()
    
    # assign directories
    i_directory = f"C:/Users/jake_/OneDrive/Desktop/financial_scraping/dow_jones_stocks/{i}" 
    competitor_directory = i_directory + "/competitors"
    
    i_csv = i_directory + '/' + i + "quarterlybalance-sheet.csv"
    i_df = pd.read_csv(i_csv)
    i_row = debt_to_equity(i_df, symbol=i)
    df = df.append(i_row)
    
    
    competitor_list =[]
    
    # iterate over files in
    # that directory
    for filename in os.listdir(competitor_directory):
        if (filename.__contains__("ratio.csv") == False):
            f = os.path.join(competitor_directory, filename)
            f = f.split("\\")[1]
            # checking if it is a file
        #     if os.path.isfile(f):
            competitor_list.append(f)
#     print(competitor_directory)
    for symbol in competitor_list:  
        competitor_csv = competitor_directory + "/" + symbol + '/' + symbol + "quarterlybalance-sheet.csv"
#         print(competitor_csv)
        competitor_df = pd.read_csv(competitor_csv)
        competitor_rows = debt_to_equity(competitor_df, symbol=symbol)
        df = df.append(competitor_rows)
        
    df = pd.DataFrame(df, columns=["items",
                                 "ticker",
                                 "Q4 2021",
                                 "Q1 2022",
                                 "Q2 2022",
                                 "Q3 2022",
                                 "Q4 2022",
                                 "Q1 2023",
                                 "Q2 2023",
                                 "Q3 2023"
                                ])
    print(df)

                   items ticker   Q4 2021    Q1 2022    Q2 2022    Q3 2022  \
32  Debt-to-Equity Ratio   AAPL       NaN        NaN   4.202777   4.787754   
25  Debt-to-Equity Ratio   AMZN  2.042056   2.065402   2.194228   2.115609   
46  Debt-to-Equity Ratio   DELL       NaN        NaN        NaN        NaN   
41  Debt-to-Equity Ratio   GOOG  0.427735   0.405868   0.390597   0.412533   
41  Debt-to-Equity Ratio  GOOGL  0.427735   0.405868   0.390597   0.412533   
44  Debt-to-Equity Ratio    HPQ       NaN -17.714777 -22.022655 -17.931406   
37  Debt-to-Equity Ratio   META  0.329183   0.332635   0.349949   0.441601   
37  Debt-to-Equity Ratio   MSFT       NaN        NaN   1.127298   1.115140   

      Q4 2022    Q1 2023   Q2 2023    Q3 2023  
32   5.961537   5.112557       NaN        NaN  
25        NaN        NaN       NaN        NaN  
46 -56.035608 -36.908205 -32.04021 -25.552321  
41        NaN        NaN       NaN        NaN  
41        NaN        NaN       NaN        NaN  
44 -14.22

                   items ticker   Q4 2021    Q1 2022    Q2 2022    Q3 2022  \
28  Debt-to-Equity Ratio    DIS       NaN   1.262430   1.233693   1.206205   
30  Debt-to-Equity Ratio   AAPL       NaN   4.299324   4.202777   4.787754   
29  Debt-to-Equity Ratio   CHTR  9.141708  10.935987  13.501265  15.162560   
34  Debt-to-Equity Ratio  CMCSA  1.871259   1.894373   1.920745   2.167132   
39  Debt-to-Equity Ratio   DISH  2.091518   2.079166   2.012932   1.884226   
25  Debt-to-Equity Ratio    FOX       NaN        NaN   1.029091   0.965714   
25  Debt-to-Equity Ratio   FOXA       NaN        NaN   1.029091   0.965714   
27  Debt-to-Equity Ratio   NFLX  1.813048   1.583846   1.429807   1.316933   
33  Debt-to-Equity Ratio   PARA  1.616731   1.561022   1.500922   1.529562   
33  Debt-to-Equity Ratio  PARAA  1.616731   1.561022   1.500922   1.529562   

     Q4 2022   Q1 2023  Q2 2023  Q3 2023  
28  1.143304       NaN      NaN      NaN  
30  5.961537       NaN      NaN      NaN  
29       NaN

                   items ticker    Q4 2021    Q1 2022    Q2 2022    Q3 2022  \
30  Debt-to-Equity Ratio    JPM  11.727723  12.832462  12.424456  12.102945   
31  Debt-to-Equity Ratio    BAC  10.736002  11.145598  10.562237  10.401408   
31  Debt-to-Equity Ratio    BMO        NaN  16.108755  14.879452  14.946771   
31  Debt-to-Equity Ratio      C  10.345201  11.109236  10.963500  10.991660   
26  Debt-to-Equity Ratio     RY        NaN  16.196242  16.722925  16.729812   
40  Debt-to-Equity Ratio     TD        NaN  16.441070  17.360721  16.943027   
36  Debt-to-Equity Ratio    WFC   9.383826   9.821672   9.596073   9.657561   

      Q4 2022  Q1 2023  Q2 2023  Q3 2023  
30        NaN      NaN      NaN      NaN  
31        NaN      NaN      NaN      NaN  
31  15.036473      NaN      NaN      NaN  
31        NaN      NaN      NaN      NaN  
26  16.741514      NaN      NaN      NaN  
40  16.215625      NaN      NaN      NaN  
36        NaN      NaN      NaN      NaN  
                   item

                   items ticker    Q4 2021    Q1 2022    Q2 2022    Q3 2022  \
26  Debt-to-Equity Ratio     PG        NaN        NaN   1.721234   1.643410   
31  Debt-to-Equity Ratio     CL  23.696223  47.981308  92.517857  25.186495   
37  Debt-to-Equity Ratio     EL        NaN        NaN   2.550659   2.473008   
35  Debt-to-Equity Ratio    JNJ   1.458938   1.387330   1.327540   1.347538   
37  Debt-to-Equity Ratio     OR   0.450153   0.517838   0.553135   0.236248   

     Q4 2022   Q1 2023  Q2 2023  Q3 2023  
26  1.515787  1.638276      NaN      NaN  
31       NaN       NaN      NaN      NaN  
37  2.740608  2.623165      NaN      NaN  
35       NaN       NaN      NaN      NaN  
37       NaN       NaN      NaN      NaN  
                   items ticker   Q4 2021   Q1 2022    Q2 2022    Q3 2022  \
26  Debt-to-Equity Ratio    TRV  3.170250  3.645020   4.096922   4.742841   
24  Debt-to-Equity Ratio    AIG  8.038025  9.251555  10.885542  12.400610   
24  Debt-to-Equity Ratio    AIZ  5.2

In [5]:
def interest_coverage_ratio(df, symbol=""):
    try:
        for j in df:
            if j == 'ticker' or j == 'items':
                continue
            df[j] = df[j].str.replace(",", "", regex=True).str.replace("(", "-", regex=True).str.replace(")", "", regex=True).str.replace("--", "", regex=True)
            df[j] = pd.to_numeric(df[j])

        df_columns = []

        for col in df.columns:
            df_columns.append(col)

        df_columns.insert(1, df_columns[-1])
        del df_columns[-1]

        first_row = df.loc[(df['ticker'] == symbol) & (df['items'] == 'Total Operating Income'), df_columns[2:]]
        second_row = df.loc[(df['ticker'] == symbol) & (df['items'] == 'Interest Expense, Suppl'), df_columns[2:]]
    #     print(first_row)
    #     print(second_row)
        third_row = first_row / second_row.values
    #     print(third_row)
        third_row['ticker'] = symbol
        third_row['items'] = "Interest Coverage Ratio"
        third_row = third_row[df_columns]
    except ValueError:
        third_row = first_row / second_row
    #     print(third_row)
        third_row['ticker'] = symbol
        third_row['items'] = "Interest Coverage Ratio"
        third_row = third_row[df_columns]
#         third_row = third_row.reshape(1,4)
    return third_row

In [6]:
for i in dow_jones_list:
    df = pd.DataFrame()
    
    # assign directories
    i_directory = f"C:/Users/jake_/OneDrive/Desktop/financial_scraping/dow_jones_stocks/{i}" 
    competitor_directory = i_directory + "/competitors"
    
    i_csv = i_directory + '/' + i + "quarterlyincome-statement.csv"
    i_df = pd.read_csv(i_csv)
    i_row = interest_coverage_ratio(i_df, symbol=i)
    df = df.append(i_row)
    
    
    competitor_list =[]
    
    # iterate over files in
    # that directory
    for filename in os.listdir(competitor_directory):
        if (filename.__contains__("ratio.csv") == False):
            f = os.path.join(competitor_directory, filename)
            f = f.split("\\")[1]
            # checking if it is a file
        #     if os.path.isfile(f):
            competitor_list.append(f)
    print(competitor_list)
    for symbol in competitor_list:
        competitor_csv = competitor_directory + "/" + symbol + '/' + symbol + "quarterlyincome-statement.csv"
#         print(competitor_csv)
        competitor_df = pd.read_csv(competitor_csv)
#             print(competitor_df)
        competitor_rows = interest_coverage_ratio(competitor_df, symbol=symbol)
        df = df.append(competitor_rows)
        
    df = pd.DataFrame(df)
    print(df)

['AMZN', 'DELL', 'GOOG', 'GOOGL', 'HPQ', 'META', 'MSFT']
                      items ticker      Q2 2022      Q3 2022    Q4 2022  \
5   Interest Coverage Ratio   AAPL    43.384949    32.094576  30.101572   
8   Interest Coverage Ratio   AMZN     5.679795     4.092382        NaN   
10  Interest Coverage Ratio   DELL          NaN          NaN   8.402985   
6   Interest Coverage Ratio   GOOG   234.373494   169.653465        NaN   
6   Interest Coverage Ratio  GOOGL   234.373494   169.653465        NaN   
10  Interest Coverage Ratio    HPQ    17.270270    13.250000   5.900000   
7   Interest Coverage Ratio   META  2089.500000  1416.000000        NaN   
6   Interest Coverage Ratio   MSFT    42.365714    40.385686  41.312500   

      Q1 2023      Q4 2021      Q1 2022   Q2 2023   Q3 2023  
5   35.908275          NaN          NaN       NaN       NaN  
8         NaN     7.178423     7.773305       NaN       NaN  
10   5.849057          NaN          NaN  4.261745  2.801471  
6         NaN   187

                      items ticker     Q1 2022     Q2 2022    Q3 2022  \
9   Interest Coverage Ratio    DIS    6.072022    3.756684   6.289474   
5   Interest Coverage Ratio   AAPL   59.780980   43.384949  32.094576   
10  Interest Coverage Ratio   CHTR    2.606604    2.964833   2.476724   
15  Interest Coverage Ratio  CMCSA    5.608258    6.577479  -2.526042   
12  Interest Coverage Ratio   DISH  110.000000  115.333333  85.400000   
9   Interest Coverage Ratio    FOX         NaN    1.752577   7.417582   
9   Interest Coverage Ratio   FOXA         NaN    1.752577   7.417582   
5   Interest Coverage Ratio   NFLX   10.489362    9.017143   8.861272   
9   Interest Coverage Ratio   PARA    2.925000    3.356522   2.450216   
9   Interest Coverage Ratio  PARAA    2.925000    3.356522   2.450216   

      Q4 2022     Q4 2021    Q1 2023  
9         NaN         NaN        NaN  
5   30.101572         NaN        NaN  
10        NaN    3.015474        NaN  
15        NaN    4.304464        NaN  
1

Empty DataFrame
Columns: [items, ticker, Q4 2021, Q1 2022, Q2 2022, Q3 2022, Q4 2022]
Index: []
['CCEP', 'COKE', 'FIZZ', 'KDP', 'KHC', 'MDLZ', 'MNST', 'PEP', 'PRMW', 'SBUX']
                      items ticker    Q4 2021    Q1 2022    Q2 2022  \
14  Interest Coverage Ratio     KO  13.806061  18.780220  11.338384   
5   Interest Coverage Ratio   CCEP        NaN        NaN        NaN   
4   Interest Coverage Ratio   COKE  10.875000  16.375000  21.000000   
6   Interest Coverage Ratio   FIZZ        NaN        NaN        NaN   
14  Interest Coverage Ratio    KDP   5.949580   4.851064   2.268571   
8   Interest Coverage Ratio    KHC   0.019868   4.603306   2.324786   
9   Interest Coverage Ratio   MDLZ  13.377778  10.604396  10.415730   
13  Interest Coverage Ratio   MNST        inf        inf        inf   
12  Interest Coverage Ratio    PEP   2.249117  21.987500   8.245763   
9   Interest Coverage Ratio   PRMW        NaN        NaN        NaN   
10  Interest Coverage Ratio   SBUX        NaN

                      items ticker    Q4 2021    Q1 2022    Q2 2022    Q3 2022
10  Interest Coverage Ratio    UNH  12.856148  16.050808  15.271949  14.461240
14  Interest Coverage Ratio     CI   5.477816   6.180602   7.594684  12.483553
11  Interest Coverage Ratio    CVS   3.519737   5.955631   7.837050  -6.945230
15  Interest Coverage Ratio    ELV   8.000000  12.572139  11.302885  11.075117
12  Interest Coverage Ratio    HUM   0.670330  14.322222  12.029703  13.843137
['AXP', 'BAC', 'C', 'COF', 'DFS', 'JPM', 'MA', 'PYPL', 'SYF', 'WFC']
                      items ticker    Q1 2022    Q2 2022    Q3 2022  \
8   Interest Coverage Ratio      V  35.641791  35.835821  37.369369   
10  Interest Coverage Ratio    AXP   8.448598   5.792711   3.089196   
11  Interest Coverage Ratio     MA  26.818182  26.473684  25.933333   
10  Interest Coverage Ratio   PYPL  12.050847  11.072464  12.850575   

      Q4 2022    Q4 2021  
8   31.993711        NaN  
10        NaN   7.897260  
11        NaN  26.17

In [7]:
def operating_margin(df, symbol=""):
#     df = pd.DataFrame()
    # for i in my_list:
#     i= 'AAPL'
#     i_csv = DATA_DIR + 'my_resources/'+ i + '/' + i + "quarterlyincome-statement.csv"
#     i_df = pd.read_csv(i_csv)
    try:
        for j in df:
            if j == 'ticker' or j == 'items':
                continue
            df[j] = df[j].str.replace(",", "", regex=True).str.replace("(", "-", regex=True).str.replace(")", "", regex=True).str.replace("--", "", regex=True)
            df[j] = pd.to_numeric(df[j])

        df_columns = []

        for col in df.columns:
            df_columns.append(col)

        df_columns.insert(1, df_columns[-1])
        del df_columns[-1]
        # print(i_df_columns)

        first_row = df.loc[(df['ticker'] == symbol) & (df['items'] == 'Total Operating Income'), df_columns[2:]]
        # gross_margin
        second_row = df.loc[(df['ticker'] == symbol) & (df['items'] == 'Total Revenue'), df_columns[2:]]
#         print(first_row)
#         print(second_row)
        third_row = first_row / second_row.values
        # print(third_row)
        third_row['ticker'] = symbol
        third_row['items'] = "Operating Margin"
        third_row = third_row[df_columns]
    except ValueError:
        third_row = first_row / second_row
    #     print(third_row)
        third_row['ticker'] = symbol
        third_row['items'] = "Operating Margin"
        third_row = third_row[df_columns]
#         third_row = third_row.reshape(1,4)
    return third_row

In [8]:
for i in dow_jones_list:
    df = pd.DataFrame()
    
    # assign directories
    i_directory = f"C:/Users/jake_/OneDrive/Desktop/financial_scraping/dow_jones_stocks/{i}" 
    competitor_directory = i_directory + "/competitors"
    
    i_csv = i_directory + '/' + i + "quarterlyincome-statement.csv"
    i_df = pd.read_csv(i_csv)
    i_row = operating_margin(i_df, symbol=i)
    df = df.append(i_row)
    
    
    competitor_list =[]
    
    # iterate over files in
    # that directory
    for filename in os.listdir(competitor_directory):
        if (filename.__contains__("ratio.csv") == False):
            f = os.path.join(competitor_directory, filename)
            f = f.split("\\")[1]
            # checking if it is a file
        #     if os.path.isfile(f):
            competitor_list.append(f)
    print(competitor_list)
    for symbol in competitor_list:
        competitor_csv = competitor_directory + "/" + symbol + '/' + symbol + "quarterlyincome-statement.csv"
#         print(competitor_csv)
        competitor_df = pd.read_csv(competitor_csv)
#             print(competitor_df)
        competitor_rows = operating_margin(competitor_df, symbol=symbol)
        df = df.append(competitor_rows)
        
    df = pd.DataFrame(df)
    print(df)

['AMZN', 'DELL', 'GOOG', 'GOOGL', 'HPQ', 'META', 'MSFT']
               items ticker   Q2 2022   Q3 2022   Q4 2022   Q1 2023   Q4 2021  \
5   Operating Margin   AAPL  0.308179  0.278162  0.276152  0.307424       NaN   
8   Operating Margin   AMZN  0.027360  0.019866       NaN       NaN  0.025180   
10  Operating Margin   DELL       NaN       NaN  0.025367  0.059351       NaN   
6   Operating Margin   GOOG  0.279156  0.248003       NaN       NaN  0.290541   
6   Operating Margin  GOOGL  0.279156  0.248003       NaN       NaN  0.290541   
10  Operating Margin    HPQ  0.077502  0.086743  0.051821       NaN       NaN   
7   Operating Margin   META  0.289987  0.204373       NaN       NaN  0.373764   
6   Operating Margin   MSFT  0.429980  0.411548  0.395083  0.428814       NaN   

     Q1 2022   Q2 2023   Q3 2023  
5        NaN       NaN       NaN  
8   0.031509       NaN       NaN  
10       NaN  0.048061  0.030824  
6   0.295452       NaN       NaN  
6   0.295452       NaN       NaN  
10 

['AMZN', 'FAST', 'GWW', 'LOW', 'SHW', 'TGT', 'TSCO', 'WMT']
               items ticker   Q4 2022   Q1 2023   Q2 2023   Q3 2023   Q4 2021  \
6   Operating Margin     HD  0.135082  0.152385  0.164642  0.158160       NaN   
8   Operating Margin   AMZN       NaN       NaN       NaN       NaN  0.025180   
4   Operating Margin   FAST       NaN       NaN       NaN       NaN  0.196475   
7   Operating Margin    GWW       NaN       NaN       NaN       NaN  0.124144   
6   Operating Margin    LOW  0.086649  0.139566  0.153916  0.039354       NaN   
13  Operating Margin    SHW       NaN       NaN       NaN       NaN  0.061319   
9   Operating Margin    TGT  0.067589  0.053476  0.012329  0.038540       NaN   
6   Operating Margin   TSCO       NaN       NaN       NaN       NaN  0.088280   
5   Operating Margin    WMT  0.043743  0.034739  0.046147  0.017636       NaN   

     Q1 2022   Q2 2022   Q3 2022  
6        NaN       NaN       NaN  
8   0.031509  0.027360  0.019866  
4   0.210094  0.215289  

               items ticker    Q4 2021   Q1 2022   Q2 2022   Q3 2022   Q3 2021
8   Operating Margin    MRK   0.264404  0.350230  0.337491  0.268200       NaN
9   Operating Margin    ABT   0.207360  0.244809  0.211069  0.170029       NaN
15  Operating Margin    BMY   0.220526  0.228194  0.217885  0.220449       NaN
11  Operating Margin    GSK   0.093944  0.318915  0.156011  0.152127       NaN
16  Operating Margin    JNJ   0.194969  0.250235  0.243131  0.244714       NaN
10  Operating Margin    LLY   0.239625  0.307810  0.186498  0.241573       NaN
14  Operating Margin   NOVN -12.354198 -6.859959 -1.345567       NaN -8.796472
12  Operating Margin    PFE   0.193611  0.384747  0.436234  0.415761       NaN
20  Operating Margin    ZTS   0.257245  0.366062  0.326511  0.333167       NaN
['AAPL', 'AVGO', 'CRM', 'CSCO', 'GOOG', 'GOOGL', 'IBM', 'ORCL', 'SAP']
               items ticker   Q2 2022   Q3 2022   Q4 2022   Q1 2023   Q1 2022  \
6   Operating Margin   MSFT  0.429980  0.411548  0.395083 

In [9]:
def net_income_margin(df, symbol=""):
#     df = pd.DataFrame()
    # for i in my_list:
#     i= 'AAPL'
#     i_csv = DATA_DIR + 'my_resources/'+ i + '/' + i + "quarterlyincome-statement.csv"
#     i_df = pd.read_csv(i_csv)
    try:
        for j in df:
            if j == 'ticker' or j == 'items':
                continue
            df[j] = df[j].str.replace(",", "", regex=True).str.replace("(", "-", regex=True).str.replace(")", "", regex=True).str.replace("--", "", regex=True)
            df[j] = pd.to_numeric(df[j])

        df_columns = []

        for col in df.columns:
            df_columns.append(col)

        df_columns.insert(1, df_columns[-1])
        del df_columns[-1]
        # print(i_df_columns)

        first_row = df.loc[(df['ticker'] == symbol) & (df['items'] == 'Total Net Income'), df_columns[2:]]
        # gross_margin
        second_row = df.loc[(df['ticker'] == symbol) & (df['items'] == 'Total Revenue'), df_columns[2:]]
#         print(first_row)
#         print(second_row)
        third_row = first_row / second_row.values
        # print(third_row)
        third_row['ticker'] = symbol
        third_row['items'] = "Net Income Margin"
        third_row = third_row[df_columns]
    except ValueError:
        third_row = first_row / second_row
    #     print(third_row)
        third_row['ticker'] = symbol
        third_row['items'] = "Net Income Margin"
        third_row = third_row[df_columns]
#         third_row = third_row.reshape(1,4)
    return third_row

In [10]:
for i in dow_jones_list:
    df = pd.DataFrame()
    
    # assign directories
    i_directory = f"C:/Users/jake_/OneDrive/Desktop/financial_scraping/dow_jones_stocks/{i}" 
    competitor_directory = i_directory + "/competitors"
    
    i_csv = i_directory + '/' + i + "quarterlyincome-statement.csv"
    i_df = pd.read_csv(i_csv)
    i_row = net_income_margin(i_df, symbol=i)
    df = df.append(i_row)
    
    
    competitor_list =[]
    
    # iterate over files in
    # that directory
    for filename in os.listdir(competitor_directory):
        if (filename.__contains__("ratio.csv") == False):
            f = os.path.join(competitor_directory, filename)
            f = f.split("\\")[1]
            # checking if it is a file
        #     if os.path.isfile(f):
            competitor_list.append(f)
    print(competitor_list)
    for symbol in competitor_list:
        competitor_csv = competitor_directory + "/" + symbol + '/' + symbol + "quarterlyincome-statement.csv"
#         print(competitor_csv)
        competitor_df = pd.read_csv(competitor_csv)
#             print(competitor_df)
        competitor_rows = net_income_margin(competitor_df, symbol=symbol)
        df = df.append(competitor_rows)
        
    df = pd.DataFrame(df)
    print(df)

['AMZN', 'DELL', 'GOOG', 'GOOGL', 'HPQ', 'META', 'MSFT']
                items ticker   Q2 2022   Q3 2022   Q4 2022   Q1 2023  \
14  Net Income Margin   AAPL  0.257098  0.234357  0.229860  0.256056   
20  Net Income Margin   AMZN -0.016728  0.022596       NaN       NaN   
23  Net Income Margin   DELL       NaN       NaN  0.000090  0.041048   
19  Net Income Margin   GOOG  0.229633  0.201326       NaN       NaN   
19  Net Income Margin  GOOGL  0.229633  0.201326       NaN       NaN   
18  Net Income Margin    HPQ  0.060643  0.076309 -0.000135       NaN   
17  Net Income Margin   META  0.232010  0.158584       NaN       NaN   
15  Net Income Margin   MSFT  0.362763  0.338898  0.322761  0.350265   

     Q4 2021   Q1 2022   Q2 2023   Q3 2023  
14       NaN       NaN       NaN       NaN  
20  0.104234 -0.033012       NaN       NaN  
23       NaN       NaN  0.019338  0.009911  
19  0.274039  0.241667       NaN       NaN  
19  0.274039  0.241667       NaN       NaN  
18       NaN  0.063777  

['BAC', 'BK', 'BLK', 'C', 'MS', 'SCHW', 'UBS', 'WFC']
                items ticker   Q4 2021   Q1 2022   Q2 2022   Q3 2022
15  Net Income Margin     GS  0.284033  0.275108  0.195381  0.166053
25  Net Income Margin    BAC       NaN       NaN       NaN       NaN
30  Net Income Margin     BK       NaN       NaN       NaN       NaN
19  Net Income Margin    BLK  0.321778  0.305597  0.237958  0.326142
34  Net Income Margin      C       NaN       NaN       NaN       NaN
13  Net Income Margin     MS  0.248956  0.240630  0.172521  0.158774
17  Net Income Margin   SCHW  0.326716  0.291597  0.340939  0.340583
27  Net Income Margin    UBS       NaN       NaN       NaN       NaN
25  Net Income Margin    WFC       NaN       NaN       NaN       NaN
['AMZN', 'FAST', 'GWW', 'LOW', 'SHW', 'TGT', 'TSCO', 'WMT']
                items ticker   Q4 2022   Q1 2023   Q2 2023   Q3 2023  \
13  Net Income Margin     HD  0.093844  0.108744  0.118127  0.111623   
20  Net Income Margin   AMZN       NaN       NaN    

                items ticker   Q4 2021   Q1 2022   Q2 2022   Q3 2022   Q4 2022
22  Net Income Margin    MCD  0.272758  0.194846  0.207765  0.337534       NaN
13  Net Income Margin    CMG  0.067823  0.078179  0.117488  0.115766       NaN
15  Net Income Margin    DPZ  0.116158  0.090010  0.095775  0.094481       NaN
23  Net Income Margin   JACK       NaN  0.113922  0.024189  0.057386  0.113856
18  Net Income Margin   PZZA  0.047259  0.018416  0.047801  0.015656       NaN
26  Net Income Margin    QSR  0.115783  0.126120  0.143990  0.208575       NaN
19  Net Income Margin   SBUX       NaN  0.101366  0.088397  0.112025  0.104350
21  Net Income Margin   SHAK -0.047871 -0.049963 -0.005148 -0.008884       NaN
27  Net Income Margin    WEN  0.109937  0.075665  0.089219  0.095685       NaN
16  Net Income Margin    YUM  0.174603  0.257919  0.136919  0.201829       NaN
22  Net Income Margin   YUMC  0.207333  0.037481  0.039004  0.076723       NaN
['DD', 'GE', 'HON', 'ITW', 'JNJ']
                it

                items ticker   Q1 2022   Q2 2022   Q3 2022   Q4 2022  \
26  Net Income Margin    WBA  0.105602  0.026188  0.008866 -0.012851   
24  Net Income Margin    ABC  0.007530  0.009494  0.006776  0.004822   
20  Net Income Margin   AMZN -0.033012 -0.016728  0.022596       NaN   
21  Net Income Margin    CVS  0.030094  0.036597 -0.042090       NaN   
20  Net Income Margin    MCK       NaN       NaN -0.000102  0.005567   
15  Net Income Margin    RAD       NaN       NaN       NaN -0.064138   

     Q4 2021   Q1 2023   Q2 2023   Q3 2023  
26       NaN       NaN       NaN       NaN  
24       NaN       NaN       NaN       NaN  
20  0.104234       NaN       NaN       NaN  
21  0.017049       NaN       NaN       NaN  
20       NaN  0.011436  0.013199       NaN  
15       NaN -0.018288 -0.056092 -0.011014  
['AMZN', 'COST', 'CVS', 'DG', 'JD', 'PDD', 'TGT', 'TJX', 'WBA']
                items ticker   Q4 2022   Q1 2023   Q2 2023   Q3 2023  \
17  Net Income Margin    WMT  0.023301  0.01

In [11]:
def acounts_receivables_turnover(i_df, j_df, symbol=""):
    """
    A high ratio may indicate that corporate collection practices are efficient with quality customers 
    who pay their debts quickly.
    
    A low ratio could be the result of inefficient collection processes, inadequate credit policies, 
    or customers who are not financially viable or creditworthy.
    """
    try:

        for j in i_df:
            if j == 'ticker' or j == 'items':
                continue
            i_df[j] = i_df[j].str.replace(",", "", regex=True).str.replace("(", "-", regex=True).str.replace(")", "", regex=True).str.replace("--", "", regex=True)
            i_df[j] = pd.to_numeric(i_df[j])

        for k in j_df:
            if k == 'ticker' or k == 'items':
                continue
            j_df[k] = j_df[k].str.replace(",", "", regex=True).str.replace("(", "-", regex=True).str.replace(")", "", regex=True).str.replace("--", "", regex=True)
            j_df[k] = pd.to_numeric(j_df[k])

        i_df_columns = []

        for col in i_df.columns:
            i_df_columns.append(col)

        i_df_columns.insert(1, i_df_columns[-1])
        del i_df_columns[-1]
        # print(i_df_columns)
        
        j_df_columns = []

        for col in j_df.columns:
            j_df_columns.append(col)

        j_df_columns.insert(1, j_df_columns[-1])
        del j_df_columns[-1]

#         df_columns = [i for i in i_df_columns if i not in j_df_columns]
        
        first_row = i_df.loc[(i_df['ticker'] == symbol) & (i_df['items'] == 'Total Revenue'), i_df_columns[2:]]
        # gross_margin
        second_row = j_df.loc[(j_df['ticker'] == symbol) & (j_df['items'] == 'Total Receivables, Net'), j_df_columns[2:]]
        # print(first_row)
        # print(second_row)
        third_row = first_row / second_row.values
        # print(third_row)
        third_row['ticker'] = symbol
        third_row['items'] = "Accounts Receivable Turnover"
        third_row = third_row[j_df_columns]

    except ValueError:
        third_row = first_row / second_row
    #     print(third_row)
        third_row['ticker'] = symbol
        third_row['items'] = "Accounts Receivable Turnover"
        third_row = third_row[j_df_columns]
#         third_row = third_row.reshape(1,4)
    return third_row

In [12]:
for i in dow_jones_list:
    df = pd.DataFrame()
    
    # assign directories
    i_directory = f"C:/Users/jake_/OneDrive/Desktop/financial_scraping/dow_jones_stocks/{i}" 
    competitor_directory = i_directory + "/competitors"
    
    i_csv = i_directory + '/' + i + "quarterlyincome-statement.csv"
    j_csv = i_directory + '/' + i + "quarterlybalance-sheet.csv"

    i_df = pd.read_csv(i_csv)
    j_df = pd.read_csv(j_csv)
    i_row = acounts_receivables_turnover(i_df, j_df, symbol=i)
    df = df.append(i_row)
    
    
    competitor_list =[]
    
    # iterate over files in
    # that directory
    for filename in os.listdir(competitor_directory):
        if (filename.__contains__("ratio.csv") == False):
            f = os.path.join(competitor_directory, filename)
            f = f.split("\\")[1]
            # checking if it is a file
        #     if os.path.isfile(f):
            competitor_list.append(f)
#     print(competitor_list)
    for symbol in competitor_list:
        i_competitor_csv = competitor_directory + "/" + symbol + '/' + symbol + "quarterlyincome-statement.csv"
        j_competitor_csv = competitor_directory + "/" + symbol + '/' + symbol + "quarterlybalance-sheet.csv"
#         print(competitor_csv)
        i_competitor_df = pd.read_csv(i_competitor_csv)
        j_competitor_df = pd.read_csv(j_competitor_csv)
#             print(competitor_df)
        competitor_rows = acounts_receivables_turnover(i_competitor_df, j_competitor_df, symbol=symbol)
        df = df.append(competitor_rows)
        
    df = pd.DataFrame(df)
    print(df)

                          items ticker   Q2 2022   Q3 2022   Q4 2022  \
0  Accounts Receivable Turnover   AAPL  2.142687  1.963898  1.479453   
0  Accounts Receivable Turnover   AMZN  3.504177  3.537462       NaN   
0  Accounts Receivable Turnover   DELL       NaN       NaN  1.232932   
0  Accounts Receivable Turnover   GOOG  1.879670  1.909885       NaN   
0  Accounts Receivable Turnover  GOOGL  1.879670  1.909885       NaN   
0  Accounts Receivable Turnover    HPQ  1.997577  1.944054  2.147874   
0  Accounts Receivable Turnover   META  2.500824  2.468513       NaN   
0  Accounts Receivable Turnover   MSFT  1.543198  1.513507  1.171799   

    Q1 2023   Q4 2021   Q1 2022  Q2 2023   Q3 2023  
0  2.162311       NaN       NaN      NaN       NaN  
0       NaN  4.177799  3.603627      NaN       NaN  
0  1.570132       NaN       NaN   1.4447  1.512358  
0       NaN  1.870499  1.909241      NaN       NaN  
0       NaN  1.870499  1.909241      NaN       NaN  
0       NaN       NaN  1.962882  

KeyError: "['Q4 2022'] not in index"

In [14]:
def inventory_turnover_ratio(i_df, j_df, symbol=""):
    try:

        for j in i_df:
            if j == 'ticker' or j == 'items':
                continue
            i_df[j] = i_df[j].str.replace(",", "", regex=True).str.replace("(", "-", regex=True).str.replace(")", "", regex=True).str.replace("--", "", regex=True)
            i_df[j] = pd.to_numeric(i_df[j])

        for k in j_df:
            if k == 'ticker' or k == 'items':
                continue
            j_df[k] = j_df[k].str.replace(",", "", regex=True).str.replace("(", "-", regex=True).str.replace(")", "", regex=True).str.replace("--", "", regex=True)
            j_df[k] = pd.to_numeric(j_df[k])

        i_df_columns = []

        for col in i_df.columns:
            i_df_columns.append(col)

        i_df_columns.insert(1, i_df_columns[-1])
        del i_df_columns[-1]
        # print(i_df_columns)
        
        j_df_columns = []

        for col in j_df.columns:
            j_df_columns.append(col)

        j_df_columns.insert(1, j_df_columns[-1])
        del j_df_columns[-1]

#         df_columns = [i for i in i_df_columns if i not in j_df_columns]
        
        first_row = i_df.loc[(i_df['ticker'] == symbol) & (i_df['items'] == 'Cost of Revenue, Total'), i_df_columns[2:]]
        # gross_margin
        second_row = j_df.loc[(j_df['ticker'] == symbol) & (j_df['items'] == 'Total Inventory'), j_df_columns[2:]]
        # print(first_row)
        # print(second_row)
        third_row = first_row / second_row.values
        # print(third_row)
        third_row['ticker'] = symbol
        third_row['items'] = "Inventory Turnover Ratio"
        third_row = third_row[j_df_columns]

    except ValueError:
        third_row = first_row / second_row
    #     print(third_row)
        third_row['ticker'] = symbol
        third_row['items'] = "Inventory Turnover Ratio"
        third_row = third_row[j_df_columns]
#         third_row = third_row.reshape(1,4)
    return third_row

In [15]:
for i in dow_jones_list:
    df = pd.DataFrame()
    
    # assign directories
    i_directory = f"C:/Users/jake_/OneDrive/Desktop/financial_scraping/dow_jones_stocks/{i}" 
    competitor_directory = i_directory + "/competitors"
    
    i_csv = i_directory + '/' + i + "quarterlyincome-statement.csv"
    j_csv = i_directory + '/' + i + "quarterlybalance-sheet.csv"

    i_df = pd.read_csv(i_csv)
    j_df = pd.read_csv(j_csv)
    i_row = inventory_turnover_ratio(i_df, j_df, symbol=i)
    df = df.append(i_row)
    
    
    competitor_list =[]
    
    # iterate over files in
    # that directory
    for filename in os.listdir(competitor_directory):
        if (filename.__contains__("ratio.csv") == False):
            f = os.path.join(competitor_directory, filename)
            f = f.split("\\")[1]
            # checking if it is a file
        #     if os.path.isfile(f):
            competitor_list.append(f)
#     print(competitor_list)
    for symbol in competitor_list:
        i_competitor_csv = competitor_directory + "/" + symbol + '/' + symbol + "quarterlyincome-statement.csv"
        j_competitor_csv = competitor_directory + "/" + symbol + '/' + symbol + "quarterlybalance-sheet.csv"
#         print(competitor_csv)
        i_competitor_df = pd.read_csv(i_competitor_csv)
        j_competitor_df = pd.read_csv(j_competitor_csv)
#             print(competitor_df)
        competitor_rows = inventory_turnover_ratio(i_competitor_df, j_competitor_df, symbol=symbol)
        df = df.append(competitor_rows)
        
    df = pd.DataFrame(df)
    print(df)

                      items ticker    Q2 2022   Q3 2022    Q4 2022   Q1 2023  \
1  Inventory Turnover Ratio   AAPL  10.021795  8.664458  10.523858  9.797947   
1  Inventory Turnover Ratio   AMZN   1.740990  1.917428        NaN       NaN   
1  Inventory Turnover Ratio   DELL        NaN       NaN   4.070024  3.239127   
1  Inventory Turnover Ratio   GOOG  15.204040  9.872624        NaN       NaN   
1  Inventory Turnover Ratio  GOOGL  15.204040  9.872624        NaN       NaN   
1  Inventory Turnover Ratio    HPQ   1.471042  1.436035   1.591047       NaN   
1  Inventory Turnover Ratio   META        NaN       NaN        NaN       NaN   
1  Inventory Turnover Ratio   MSFT   5.617754  4.737561   4.390433  3.620431   

     Q4 2021    Q1 2022   Q2 2023   Q3 2023  
1        NaN        NaN       NaN       NaN  
1   2.537837   1.900677       NaN       NaN  
1        NaN        NaN  3.567228  3.080687  
1  28.194872  21.620891       NaN       NaN  
1  28.194872  21.620891       NaN       NaN  
1  

KeyError: "['Q4 2022'] not in index"

In [16]:
def current_ratio(df, symbol=""):
    """
    A ratio under 1.00 indicates that the company’s debts due in a year or less 
    are greater than its assets—cash or other short-term assets expected to be converted to cash within a year or less. 
    
    A current ratio of less than 1.00 may seem alarming, although different situations 
    can negatively affect the current ratio in a solid company.
    """
    
    try:
        for j in df:
            if j == 'ticker' or j == 'items':
                continue
            df[j] = df[j].str.replace(",", "", regex=True).str.replace("(", "-", regex=True).str.replace(")", "", regex=True).str.replace("--", "", regex=True)
            df[j] = pd.to_numeric(df[j])

        df_columns = []

        for col in df.columns:
            df_columns.append(col)

        df_columns.insert(1, df_columns[-1])
        del df_columns[-1]
        # print(i_df_columns)

        first_row = df.loc[(df['ticker'] == symbol) & (df['items'] == 'Total Current Assets'), df_columns[2:]]
        # gross_margin
        second_row = df.loc[(df['ticker'] == symbol) & (df['items'] == 'Total Current Liabilities'), df_columns[2:]]
#         print(first_row)
#         print(second_row)
        third_row = first_row / second_row.values
        # print(third_row)
        third_row['ticker'] = symbol
        third_row['items'] = "Current Ratio"
        third_row = third_row[df_columns]
    except ValueError:
        third_row = first_row / second_row
    #     print(third_row)
        third_row['ticker'] = symbol
        third_row['items'] = "Current Ratio"
        third_row = third_row[df_columns]
#         third_row = third_row.reshape(1,4)
    return third_row

In [17]:
for i in dow_jones_list:
    df = pd.DataFrame()
    
    # assign directories
    i_directory = f"C:/Users/jake_/OneDrive/Desktop/financial_scraping/dow_jones_stocks/{i}" 
    competitor_directory = i_directory + "/competitors"
    
    i_csv = i_directory + '/' + i + "quarterlybalance-sheet.csv"
    i_df = pd.read_csv(i_csv)
    i_row = current_ratio(i_df, symbol=i)
    df = df.append(i_row)
    
    
    competitor_list =[]
    
    # iterate over files in
    # that directory
    for filename in os.listdir(competitor_directory):
        if (filename.__contains__("ratio.csv") == False):
            f = os.path.join(competitor_directory, filename)
            f = f.split("\\")[1]
            # checking if it is a file
        #     if os.path.isfile(f):
            competitor_list.append(f)
    print(competitor_list)
    for symbol in competitor_list:
        competitor_csv = competitor_directory + "/" + symbol + '/' + symbol + "quarterlybalance-sheet.csv"
#         print(competitor_csv)
        competitor_df = pd.read_csv(competitor_csv)
#             print(competitor_df)
        competitor_rows = current_ratio(competitor_df, symbol=symbol)
        df = df.append(competitor_rows)
        
    df = pd.DataFrame(df)
    print(df)

['AMZN', 'DELL', 'GOOG', 'GOOGL', 'HPQ', 'META', 'MSFT']
            items ticker   Q2 2022   Q3 2022   Q4 2022   Q1 2023   Q4 2021  \
11  Current Ratio   AAPL  0.926844  0.864629  0.879356  0.938020       NaN   
8   Current Ratio   AMZN  0.952784  0.936593       NaN       NaN  1.135760   
14  Current Ratio   DELL       NaN       NaN  0.801028  0.779811       NaN   
10  Current Ratio   GOOG  2.809450  2.517604       NaN       NaN  2.928113   
10  Current Ratio  GOOGL  2.809450  2.517604       NaN       NaN  2.928113   
12  Current Ratio    HPQ  0.813499  0.857772  0.757122       NaN       NaN   
7   Current Ratio   META  2.520007  2.570415       NaN       NaN  3.154294   
11  Current Ratio   MSFT  2.247297  1.987655  1.784607  1.840186       NaN   

     Q1 2022   Q2 2023   Q3 2023  
11       NaN       NaN       NaN  
8   0.959630       NaN       NaN  
14       NaN  0.771359  0.745803  
10  2.871005       NaN       NaN  
10  2.871005       NaN       NaN  
12  0.742806       NaN       N

['BAC', 'BK', 'BLK', 'C', 'MS', 'SCHW', 'UBS', 'WFC']
           items ticker  Q4 2021  Q1 2022  Q2 2022  Q3 2022
6  Current Ratio     GS      NaN      NaN      NaN      NaN
2  Current Ratio    BLK      NaN      NaN      NaN      NaN
4  Current Ratio     MS      NaN      NaN      NaN      NaN
2  Current Ratio   SCHW      NaN      NaN      NaN      NaN
['AMZN', 'FAST', 'GWW', 'LOW', 'SHW', 'TGT', 'TSCO', 'WMT']
           items ticker   Q4 2022   Q1 2023   Q2 2023   Q3 2023   Q4 2021  \
4  Current Ratio     HD  1.012616  1.114523  1.183481  1.387191       NaN   
8  Current Ratio   AMZN       NaN       NaN       NaN       NaN  1.135760   
7  Current Ratio   FAST       NaN       NaN       NaN       NaN  4.189150   
6  Current Ratio    GWW       NaN       NaN       NaN       NaN  2.625000   
5  Current Ratio    LOW  1.019931  1.173148  1.112982  1.197116       NaN   
8  Current Ratio    SHW       NaN       NaN       NaN       NaN  0.883566   
5  Current Ratio    TGT  0.991999  0.866290  0.

            items ticker   Q4 2021   Q1 2022   Q2 2022   Q3 2022
12  Current Ratio    MMM  1.704815  1.579108  1.466653  1.560830
13  Current Ratio     DD  2.900629  2.819797  2.549327  2.386099
13  Current Ratio     GE  1.277077  1.186619  1.125735  1.099255
13  Current Ratio    HON  1.300595  1.212413  1.195518  1.243857
9   Current Ratio    ITW  1.836888  1.731985  1.483698  1.410747
12  Current Ratio    JNJ  1.348317  1.392579  1.424489  1.432405
['ABT', 'BMY', 'GSK', 'JNJ', 'LLY', 'NOVN', 'PFE', 'SAN', 'ZTS']
            items ticker   Q4 2021   Q1 2022   Q2 2022   Q3 2022   Q3 2021
12  Current Ratio    MRK  1.267845  1.397383  1.386222  1.459301       NaN
11  Current Ratio    ABT  1.849599  1.852534  2.013880  1.858960       NaN
16  Current Ratio    BMY  1.521035  1.319881  1.443270  1.415531       NaN
10  Current Ratio    GSK  0.788931  1.194003  1.432219  0.966401       NaN
12  Current Ratio    JNJ  1.348317  1.392579  1.424489  1.432405       NaN
10  Current Ratio    LLY  1.22

In [18]:
def quick_ratio(df, symbol=""):
    """
    The higher the ratio result, the better a company's liquidity and financial health; 
    the lower the ratio, the more likely the company will struggle with paying debts.
    """
    try:
        for j in df:
            if j == 'ticker' or j == 'items':
                continue
            df[j] = df[j].str.replace(",", "", regex=True).str.replace("(", "-", regex=True).str.replace(")", "", regex=True).str.replace("--", "", regex=True)
            df[j] = pd.to_numeric(df[j])

        df_columns = []

        for col in df.columns:
            df_columns.append(col)

        df_columns.insert(1, df_columns[-1])
        del df_columns[-1]
        # print(i_df_columns)

        first_row = df.loc[(df['ticker'] == symbol) & (df['items'] == 'Total Current Assets'), df_columns[2:]]
        # gross_margin
        second_row = df.loc[(df['ticker'] == symbol) & (df['items'] == 'Total Inventory'), df_columns[2:]]
#         print(first_row)
#         print(second_row)
        third_row = first_row / second_row.values
        # print(third_row)
        third_row['ticker'] = symbol
        third_row['items'] = "Quick Ratio"
        third_row = third_row[df_columns]
    except ValueError:
        third_row = first_row / second_row
    #     print(third_row)
        third_row['ticker'] = symbol
        third_row['items'] = "Quick Ratio"
        third_row = third_row[df_columns]
#         third_row = third_row.reshape(1,4)
    return third_row

In [19]:
for i in dow_jones_list:
    df = pd.DataFrame()
    
    # assign directories
    i_directory = f"C:/Users/jake_/OneDrive/Desktop/financial_scraping/dow_jones_stocks/{i}" 
    competitor_directory = i_directory + "/competitors"
    
    i_csv = i_directory + '/' + i + "quarterlybalance-sheet.csv"
    i_df = pd.read_csv(i_csv)
    i_row = quick_ratio(i_df, symbol=i)
    df = df.append(i_row)
    
    
    competitor_list =[]
    
    # iterate over files in
    # that directory
    for filename in os.listdir(competitor_directory):
        if (filename.__contains__("ratio.csv") == False):
            f = os.path.join(competitor_directory, filename)
            f = f.split("\\")[1]
            # checking if it is a file
        #     if os.path.isfile(f):
            competitor_list.append(f)
    print(competitor_list)
    for symbol in competitor_list:
        competitor_csv = competitor_directory + "/" + symbol + '/' + symbol + "quarterlybalance-sheet.csv"
#         print(competitor_csv)
        competitor_df = pd.read_csv(competitor_csv)
#             print(competitor_df)
        competitor_rows = quick_ratio(competitor_df, symbol=symbol)
        df = df.append(competitor_rows)
        
    df = pd.DataFrame(df)
    print(df)

['AMZN', 'DELL', 'GOOG', 'GOOGL', 'HPQ', 'META', 'MSFT']
          items ticker    Q2 2022    Q3 2022    Q4 2022    Q1 2023  \
11  Quick Ratio   AAPL  21.644689  20.668507  27.376668  18.882258   
8   Quick Ratio   AMZN   3.503447   3.587279        NaN        NaN   
14  Quick Ratio   DELL        NaN        NaN   7.635300   6.591684   
10  Quick Ratio   GOOG  87.056061  52.632763        NaN        NaN   
10  Quick Ratio  GOOGL  87.056061  52.632763        NaN        NaN   
12  Quick Ratio    HPQ   2.629137   2.807129   2.607110        NaN   
7   Quick Ratio   META        NaN        NaN        NaN        NaN   
11  Quick Ratio   MSFT  57.697251  46.699636  45.345804  37.678538   

       Q4 2021     Q1 2022  Q2 2023   Q3 2023  
11         NaN         NaN      NaN       NaN  
8     4.950368    3.826450      NaN       NaN  
14         NaN         NaN  7.18375  6.284349  
10  160.805983  129.914536      NaN       NaN  
10  160.805983  129.914536      NaN       NaN  
12         NaN    2.4874

         items ticker  Q4 2021  Q1 2022  Q2 2022  Q3 2022
6  Quick Ratio     GS      NaN      NaN      NaN      NaN
2  Quick Ratio    BLK      NaN      NaN      NaN      NaN
4  Quick Ratio     MS      NaN      NaN      NaN      NaN
2  Quick Ratio   SCHW      NaN      NaN      NaN      NaN
['AMZN', 'FAST', 'GWW', 'LOW', 'SHW', 'TGT', 'TSCO', 'WMT']
         items ticker   Q4 2022   Q1 2023   Q2 2023   Q3 2023   Q4 2021  \
4  Quick Ratio     HD  1.316612  1.338775  1.262688  1.309577       NaN   
8  Quick Ratio   AMZN       NaN       NaN       NaN       NaN  4.950368   
7  Quick Ratio   FAST       NaN       NaN       NaN       NaN  1.874672   
6  Quick Ratio    GWW       NaN       NaN       NaN       NaN  2.144920   
5  Quick Ratio    LOW  1.139449  1.265428  1.172694  1.261089       NaN   
8  Quick Ratio    SHW       NaN       NaN       NaN       NaN  2.622730   
5  Quick Ratio    TGT  1.551791  1.190280  1.204504  1.191389       NaN   
4  Quick Ratio   TSCO       NaN       NaN       Na

          items ticker    Q4 2021    Q1 2022    Q2 2022    Q3 2022  Q3 2021
12  Quick Ratio    MRK   5.084159   5.400762   5.802349   5.978090      NaN
11  Quick Ratio    ABT   4.700213   4.116851   4.230548   4.332926      NaN
16  Quick Ratio    BMY  15.876850  14.316065  14.092437  12.919961      NaN
10  Quick Ratio    GSK   3.229120   4.828919  11.618353   4.346212      NaN
12  Quick Ratio    JNJ   5.870704   5.498089   5.582495   5.587666      NaN
10  Quick Ratio    LLY   4.748327   4.357822   4.389587   4.604542      NaN
9   Quick Ratio   NOVN        NaN  39.703654  39.804362        NaN      NaN
14  Quick Ratio    PFE   6.589359   5.453452   6.453606   7.400715      NaN
11  Quick Ratio    ZTS   3.603744   3.323772   3.016327   2.844551      NaN
['AAPL', 'AVGO', 'CRM', 'CSCO', 'GOOG', 'GOOGL', 'IBM', 'ORCL', 'SAP']
          items ticker    Q2 2022    Q3 2022    Q4 2022    Q1 2023  \
11  Quick Ratio   MSFT  57.697251  46.699636  45.345804  37.678538   
9   Quick Ratio   AAPL  21.64

In [20]:
def return_on_assets(i_df, j_df, symbol=""):
    try:

        for j in i_df:
            if j == 'ticker' or j == 'items':
                continue
            i_df[j] = i_df[j].str.replace(",", "", regex=True).str.replace("(", "-", regex=True).str.replace(")", "", regex=True).str.replace("--", "", regex=True)
            i_df[j] = pd.to_numeric(i_df[j])

        for k in j_df:
            if k == 'ticker' or k == 'items':
                continue
            j_df[k] = j_df[k].str.replace(",", "", regex=True).str.replace("(", "-", regex=True).str.replace(")", "", regex=True).str.replace("--", "", regex=True)
            j_df[k] = pd.to_numeric(j_df[k])

        i_df_columns = []

        for col in i_df.columns:
            i_df_columns.append(col)

        i_df_columns.insert(1, i_df_columns[-1])
        del i_df_columns[-1]
        # print(i_df_columns)
        
        j_df_columns = []

        for col in j_df.columns:
            j_df_columns.append(col)

        j_df_columns.insert(1, j_df_columns[-1])
        del j_df_columns[-1]

#         df_columns = [i for i in i_df_columns if i not in j_df_columns]
        
        first_row = i_df.loc[(i_df['ticker'] == symbol) & (i_df['items'] == 'Income After Tax'), i_df_columns[2:]]
        # gross_margin
        second_row = j_df.loc[(j_df['ticker'] == symbol) & (j_df['items'] == 'Total Assets'), j_df_columns[2:]]
        # print(first_row)
        # print(second_row)
        third_row = first_row / second_row.values
        # print(third_row)
        third_row['ticker'] = symbol
        third_row['items'] = "Return on Assets"
        third_row = third_row[j_df_columns]

    except ValueError:
        third_row = first_row / second_row
    #     print(third_row)
        third_row['ticker'] = symbol
        third_row['items'] = "Return on Assets"
        third_row = third_row[j_df_columns]
#         third_row = third_row.reshape(1,4)
    return third_row

In [21]:
for i in dow_jones_list:
    df = pd.DataFrame()
    
    # assign directories
    i_directory = f"C:/Users/jake_/OneDrive/Desktop/financial_scraping/dow_jones_stocks/{i}" 
    competitor_directory = i_directory + "/competitors"
    
    i_csv = i_directory + '/' + i + "quarterlyincome-statement.csv"
    j_csv = i_directory + '/' + i + "quarterlybalance-sheet.csv"

    i_df = pd.read_csv(i_csv)
    j_df = pd.read_csv(j_csv)
    i_row = return_on_assets(i_df, j_df, symbol=i)
    df = df.append(i_row)
    
    
    competitor_list =[]
    
    # iterate over files in
    # that directory
    for filename in os.listdir(competitor_directory):
        if (filename.__contains__("ratio.csv") == False):
            f = os.path.join(competitor_directory, filename)
            f = f.split("\\")[1]
            # checking if it is a file
        #     if os.path.isfile(f):
            competitor_list.append(f)
#     print(competitor_list)
    for symbol in competitor_list:
        i_competitor_csv = competitor_directory + "/" + symbol + '/' + symbol + "quarterlyincome-statement.csv"
        j_competitor_csv = competitor_directory + "/" + symbol + '/' + symbol + "quarterlybalance-sheet.csv"
#         print(competitor_csv)
        i_competitor_df = pd.read_csv(i_competitor_csv)
        j_competitor_df = pd.read_csv(j_competitor_csv)
#             print(competitor_df)
        competitor_rows = return_on_assets(i_competitor_df, j_competitor_df, symbol=symbol)
        df = df.append(competitor_rows)
        
    df = pd.DataFrame(df)
    print(df)

               items ticker   Q2 2022   Q3 2022   Q4 2022   Q1 2023   Q4 2021  \
12  Return on Assets   AAPL  0.071322  0.057810  0.058740  0.086513       NaN   
17  Return on Assets   AMZN -0.004803  0.006712       NaN       NaN  0.034055   
19  Return on Assets   DELL       NaN       NaN -0.008239  0.012092       NaN   
17  Return on Assets   GOOG  0.045053  0.038827       NaN       NaN  0.057456   
17  Return on Assets  GOOGL  0.045053  0.038827       NaN       NaN  0.057456   
16  Return on Assets    HPQ  0.025062  0.028512 -0.000052       NaN       NaN   
15  Return on Assets   META  0.039386  0.024568       NaN       NaN  0.061963   
13  Return on Assets   MSFT  0.055128  0.048542  0.045883  0.048796       NaN   

     Q1 2022  Q2 2023  Q3 2023  
12       NaN      NaN      NaN  
17 -0.009356      NaN      NaN  
19       NaN   0.0057  0.00283  
17  0.046027      NaN      NaN  
17  0.046027      NaN      NaN  
16  0.027909      NaN      NaN  
15  0.045458      NaN      NaN  
13    

KeyError: "['Q4 2022'] not in index"

In [22]:
def return_on_equity(i_df, j_df, symbol=""):
    try:

        for j in i_df:
            if j == 'ticker' or j == 'items':
                continue
            i_df[j] = i_df[j].str.replace(",", "", regex=True).str.replace("(", "-", regex=True).str.replace(")", "", regex=True).str.replace("--", "", regex=True)
            i_df[j] = pd.to_numeric(i_df[j])

        for k in j_df:
            if k == 'ticker' or k == 'items':
                continue
            j_df[k] = j_df[k].str.replace(",", "", regex=True).str.replace("(", "-", regex=True).str.replace(")", "", regex=True).str.replace("--", "", regex=True)
            j_df[k] = pd.to_numeric(j_df[k])

        i_df_columns = []

        for col in i_df.columns:
            i_df_columns.append(col)

        i_df_columns.insert(1, i_df_columns[-1])
        del i_df_columns[-1]
        # print(i_df_columns)
        
        j_df_columns = []

        for col in j_df.columns:
            j_df_columns.append(col)

        j_df_columns.insert(1, j_df_columns[-1])
        del j_df_columns[-1]

#         df_columns = [i for i in i_df_columns if i not in j_df_columns]
        
        first_row = i_df.loc[(i_df['ticker'] == symbol) & (i_df['items'] == 'Income After Tax'), i_df_columns[2:]]
        # gross_margin
        second_row = j_df.loc[(j_df['ticker'] == symbol) & (j_df['items'] == 'Total Equity'), j_df_columns[2:]]
        # print(first_row)
        # print(second_row)
        third_row = first_row / second_row.values
        # print(third_row)
        third_row['ticker'] = symbol
        third_row['items'] = "Return on Equity"
        third_row = third_row[j_df_columns]

    except ValueError:
        third_row = first_row / second_row
    #     print(third_row)
        third_row['ticker'] = symbol
        third_row['items'] = "Return on Equity"
        third_row = third_row[j_df_columns]
#         third_row = third_row.reshape(1,4)
    return third_row

In [23]:
for i in dow_jones_list:
    df = pd.DataFrame()
    
    # assign directories
    i_directory = f"C:/Users/jake_/OneDrive/Desktop/financial_scraping/dow_jones_stocks/{i}" 
    competitor_directory = i_directory + "/competitors"
    
    i_csv = i_directory + '/' + i + "quarterlyincome-statement.csv"
    j_csv = i_directory + '/' + i + "quarterlybalance-sheet.csv"

    i_df = pd.read_csv(i_csv)
    j_df = pd.read_csv(j_csv)
    i_row = return_on_equity(i_df, j_df, symbol=i)
    df = df.append(i_row)
    
    
    competitor_list =[]
    
    # iterate over files in
    # that directory
    for filename in os.listdir(competitor_directory):
        if (filename.__contains__("ratio.csv") == False):
            f = os.path.join(competitor_directory, filename)
            f = f.split("\\")[1]
            # checking if it is a file
        #     if os.path.isfile(f):
            competitor_list.append(f)
#     print(competitor_list)
    for symbol in competitor_list:
        i_competitor_csv = competitor_directory + "/" + symbol + '/' + symbol + "quarterlyincome-statement.csv"
        j_competitor_csv = competitor_directory + "/" + symbol + '/' + symbol + "quarterlybalance-sheet.csv"
#         print(competitor_csv)
        i_competitor_df = pd.read_csv(i_competitor_csv)
        j_competitor_df = pd.read_csv(j_competitor_csv)
#             print(competitor_df)
        competitor_rows = return_on_equity(i_competitor_df, j_competitor_df, symbol=symbol)
        df = df.append(competitor_rows)
        
    df = pd.DataFrame(df)
    print(df)

               items ticker   Q2 2022   Q3 2022   Q4 2022   Q1 2023   Q4 2021  \
12  Return on Equity   AAPL  0.371074  0.334590  0.408924  0.528813       NaN   
17  Return on Equity   AMZN -0.015342  0.020911       NaN       NaN  0.103599   
19  Return on Equity   DELL       NaN       NaN  0.453412 -0.434200       NaN   
17  Return on Equity   GOOG  0.062650  0.054845       NaN       NaN  0.082032   
17  Return on Equity  GOOGL  0.062650  0.054845       NaN       NaN  0.082032   
16  Return on Equity    HPQ -0.526870 -0.482744  0.000685       NaN       NaN   
15  Return on Equity   META  0.053170  0.035417       NaN       NaN  0.082360   
13  Return on Equity   MSFT  0.117274  0.102674  0.100515  0.101149       NaN   

     Q1 2022   Q2 2023   Q3 2023  
12       NaN       NaN       NaN  
17 -0.028679       NaN       NaN  
19       NaN -0.176923 -0.069472  
17  0.064708       NaN       NaN  
17  0.064708       NaN       NaN  
16 -0.466495       NaN       NaN  
15  0.060579       NaN   

KeyError: "['Q4 2022'] not in index"